In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import soundfile as sf
import tempfile
import os
import io
import base64
import numpy as np
import json
from pathlib import Path
from fish_speech_tts import FishSpeechTTS

class FishSpeechTTSApp:
    def __init__(self):
        """Fish Speech TTSのためのインタラクティブなJupyter UIを初期化します"""
        # TTSエンジンを初期化
        self.tts = None
        self.audio_data = None
        self.sample_rate = None
        self.output_filename = "generated_audio.wav"
        
        # サンプル話者IDのリスト - 実際の利用可能なIDはモデルに依存します
        self.speaker_ids = {
            "デフォルト": None,
            "男性1": "male_speaker_1",
            "女性1": "female_speaker_1",
            "子供": "child_speaker_1",
            "老人": "elder_speaker_1",
            "ナレーター": "narrator_1",
        }
        
        # モデル設定セクション
        self.llama_path = widgets.Text(
            value="checkpoints/fish-speech-1.5",
            description="LLaMAモデルパス:",
            layout=widgets.Layout(width='90%')
        )
        
        self.decoder_path = widgets.Text(
            value="checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth",
            description="デコーダパス:",
            layout=widgets.Layout(width='90%')
        )
        
        self.device = widgets.Dropdown(
            options=['cuda', 'cpu', 'mps'],
            value='cuda',
            description='デバイス:',
        )
        
        self.half_precision = widgets.Checkbox(
            value=False,
            description='半精度(FP16)を使用',
        )
        
        self.compile_model = widgets.Checkbox(
            value=False,
            description='モデルをコンパイル',
        )
        
        self.model_init_btn = widgets.Button(
            description='モデルを初期化',
            button_style='success',
            icon='rocket'
        )
        self.model_init_btn.on_click(self.initialize_model)
        
        self.model_status = widgets.HTML(
            value="<b>ステータス:</b> モデルは初期化されていません"
        )
        
        # テキスト入力セクション
        self.text_input = widgets.Textarea(
            value="こんにちは、これはFish Speechのテキスト読み上げデモです。",
            placeholder='ここにテキストを入力してください',
            description='入力テキスト:',
            layout=widgets.Layout(width='90%', height='150px')
        )
        
        # テキストファイルアップロード機能
        self.text_upload = widgets.FileUpload(
            accept='.txt,.md,.html',
            multiple=False,
            description='テキストをアップロード',
            icon='file-upload'
        )
        self.text_upload.observe(self.handle_text_upload, names='value')
        
        # パラメータセクション
        self.max_new_tokens = widgets.IntSlider(
            value=1024,
            min=0,
            max=2048,
            step=8,
            description='最大トークン数:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        )
        
        self.chunk_length = widgets.IntSlider(
            value=200,
            min=100,
            max=300,
            step=8,
            description='チャンク長:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        )
        
        self.top_p = widgets.FloatSlider(
            value=0.7,
            min=0.1,
            max=1.0,
            step=0.01,
            description='Top-p:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f'
        )
        
        self.repetition_penalty = widgets.FloatSlider(
            value=1.2,
            min=1.0,
            max=2.0,
            step=0.01,
            description='繰り返しペナルティ:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f'
        )
        
        self.temperature = widgets.FloatSlider(
            value=0.7,
            min=0.1,
            max=1.0,
            step=0.01,
            description='温度:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f'
        )
        
        self.seed = widgets.IntText(
            value=0,
            description='シード:',
            disabled=False,
            layout=widgets.Layout(width='300px')
        )
        self.use_seed = widgets.Checkbox(
            value=False,
            description='シードを使用',
        )
        
        # 読み上げ設定セクション
        self.reading_speed = widgets.FloatSlider(
            value=1.0,
            min=0.5,
            max=2.0,
            step=0.1,
            description='読み上げ速度:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f'
        )
        
        self.pitch_adjustment = widgets.FloatSlider(
            value=0.0,
            min=-10.0,
            max=10.0,
            step=0.5,
            description='ピッチ調整:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f'
        )
        
        self.energy_scale = widgets.FloatSlider(
            value=1.0,
            min=0.5,
            max=2.0,
            step=0.1,
            description='音量スケール:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f'
        )
        
        # 参照音声セクション
        self.voice_selection_method = widgets.RadioButtons(
            options=['デフォルト', '話者ID', '参照音声'],
            value='デフォルト',
            description='音声選択方法:',
            disabled=False,
            layout=widgets.Layout(width='50%')
        )
        self.voice_selection_method.observe(self.handle_voice_selection_change, names='value')
        
        self.speaker_id_dropdown = widgets.Dropdown(
            options=list(self.speaker_ids.keys()),
            value='デフォルト',
            description='話者ID:',
            disabled=False,
            layout=widgets.Layout(width='50%')
        )
        
        self.reference_upload = widgets.FileUpload(
            accept='.wav,.mp3,.ogg,.flac',
            multiple=False,
            description='参照音声アップロード',
            icon='file-upload'
        )
        
        self.reference_text = widgets.Text(
            value="",
            placeholder='参照音声のテキスト内容を入力してください',
            description='参照テキスト:',
            layout=widgets.Layout(width='90%')
        )
        
        # 生成ボタンと出力セクション
        self.generate_btn = widgets.Button(
            description='音声を生成',
            button_style='primary',
            icon='play',
            disabled=True
        )
        self.generate_btn.on_click(self.generate_audio)
        
        self.save_btn = widgets.Button(
            description='音声を保存',
            icon='save',
            disabled=True
        )
        self.save_btn.on_click(self.save_audio)
        
        self.download_btn = widgets.Button(
            description='音声をダウンロード',
            icon='download',
            disabled=True
        )
        self.download_btn.on_click(self.download_audio)
        
        self.play_btn = widgets.Button(
            description='最後の音声を再生',
            icon='volume-up',
            disabled=True
        )
        self.play_btn.on_click(self.play_audio)
        
        self.filename_input = widgets.Text(
            value=self.output_filename,
            placeholder='ファイル名.wav',
            description='ファイル名:',
            layout=widgets.Layout(width='300px')
        )
        
        self.output_widget = widgets.Output()
        self.audio_widget = widgets.Output()
        self.download_link_widget = widgets.HTML(value="")
        self.status_widget = widgets.HTML(
            value="<i>音声をまだ生成していません</i>"
        )
        
        # UIのレイアウト作成
        self.create_layout()
        
    def create_layout(self):
        """ウィジェットのレイアウトを作成します"""
        # タブ作成
        self.tab = widgets.Tab()
        
        # モデル設定タブ
        model_setup_tab = widgets.VBox([
            widgets.HTML("<h3>モデル設定</h3>"),
            self.llama_path,
            self.decoder_path,
            widgets.HBox([self.device, self.half_precision, self.compile_model]),
            self.model_init_btn,
            self.model_status
        ])
        
        # 主なTTSタブ
        main_tts_tab = widgets.VBox([
            widgets.HTML("<h3>テキスト読み上げ</h3>"),
            self.text_upload,
            self.text_input,
            widgets.HTML("<h4>パラメータ設定</h4>"),
            widgets.HBox([self.max_new_tokens, self.chunk_length]),
            widgets.HBox([self.top_p, self.temperature]),
            widgets.HBox([self.repetition_penalty]),
            widgets.HBox([self.use_seed, self.seed]),
            widgets.HTML("<h4>操作</h4>"),
            widgets.HBox([self.generate_btn, self.play_btn]),
            widgets.HBox([self.save_btn, self.filename_input, self.download_btn]),
            self.status_widget,
            self.download_link_widget,
            self.audio_widget,
            self.output_widget
        ])
        
        # 音声設定タブ
        voice_settings_tab = widgets.VBox([
            widgets.HTML("<h3>音声設定</h3>"),
            widgets.HTML("<h4>読み上げ設定</h4>"),
            self.reading_speed,
            self.pitch_adjustment,
            self.energy_scale,
            widgets.HTML("<p><i>注意: これらの設定はモデルの能力に依存します</i></p>"),
            widgets.HTML("<h4>音声選択</h4>"),
            self.voice_selection_method,
            widgets.HBox([
                widgets.VBox([
                    self.speaker_id_dropdown,
                ], layout=widgets.Layout(display='none', width='100%')),
                widgets.VBox([
                    self.reference_upload,
                    self.reference_text,
                ], layout=widgets.Layout(display='none', width='100%'))
            ], layout=widgets.Layout(width='100%'))
        ])
        
        # タブに追加
        self.tab.children = [model_setup_tab, main_tts_tab, voice_settings_tab]
        self.tab.set_title(0, 'モデル設定')
        self.tab.set_title(1, 'テキスト読み上げ')
        self.tab.set_title(2, '音声設定')
        
        # 全体表示
        display(self.tab)
    
    def handle_voice_selection_change(self, change):
        """音声選択方法が変更されたときの処理"""
        method = change['new']
        
        # タブの子ウィジェットを取得
        voice_tab = self.tab.children[2]
        voice_selection_box = voice_tab.children[6]
        
        # 子コンテナを取得
        speaker_id_container = voice_selection_box.children[0]
        reference_container = voice_selection_box.children[1]
        
        # 選択された方法に基づいて表示/非表示を切り替え
        if method == '話者ID':
            speaker_id_container.layout.display = 'block'
            reference_container.layout.display = 'none'
        elif method == '参照音声':
            speaker_id_container.layout.display = 'none'
            reference_container.layout.display = 'block'
        else:  # デフォルト
            speaker_id_container.layout.display = 'none'
            reference_container.layout.display = 'none'
    
    def handle_text_upload(self, change):
        """テキストファイルがアップロードされたときの処理"""
        if not change.new:
            return
            
        # 最初のファイルのみ処理
        uploaded_file = next(iter(change.new.values()))
        
        try:
            # ファイルの内容をデコード
            content = uploaded_file['content'].decode('utf-8')
            # テキスト入力欄に設定
            self.text_input.value = content
        except UnicodeDecodeError:
            self.status_widget.value = "<span style='color:red'>エラー: UTF-8でデコードできないファイルです</span>"
    
    def initialize_model(self, btn):
        """TTSモデルを初期化します"""
        self.model_status.value = "<b>ステータス:</b> モデルを初期化中..."
        
        try:
            self.tts = FishSpeechTTS(
                llama_checkpoint_path=self.llama_path.value,
                decoder_checkpoint_path=self.decoder_path.value,
                device=self.device.value,
                half=self.half_precision.value,
                compile=self.compile_model.value
            )
            self.model_status.value = "<b>ステータス:</b> <span style='color:green'>モデルの初期化に成功しました</span>"
            self.generate_btn.disabled = False
        except Exception as e:
            self.model_status.value = f"<b>ステータス:</b> <span style='color:red'>エラー: {str(e)}</span>"
            self.generate_btn.disabled = True
    
    def generate_audio(self, btn):
        """テキストから音声を生成します"""
        if self.tts is None:
            self.status_widget.value = "<span style='color:red'>エラー: 先にモデルを初期化してください</span>"
            return
        
        text = self.text_input.value
        if not text:
            self.status_widget.value = "<span style='color:red'>エラー: テキストを入力してください</span>"
            return
        
        self.status_widget.value = "<b>ステータス:</b> 音声を生成中..."
        
        # パラメータ設定
        params = {
            'text': text,
            'max_new_tokens': self.max_new_tokens.value,
            'chunk_length': self.chunk_length.value,
            'top_p': self.top_p.value,
            'repetition_penalty': self.repetition_penalty.value,
            'temperature': self.temperature.value,
            'seed': self.seed.value if self.use_seed.value else None,
        }
        
        # 音声選択方法に基づいて設定
        method = self.voice_selection_method.value
        if method == '話者ID':
            selected_speaker = self.speaker_id_dropdown.value
            if selected_speaker != 'デフォルト':
                params['reference_id'] = self.speaker_ids[selected_speaker]
        elif method == '参照音声' and self.reference_upload.value:
            # アップロードされたファイルの最初のファイルを使用
            uploaded_file = list(self.reference_upload.value.values())[0]
            params['reference_audio'] = uploaded_file['content']
            params['reference_text'] = self.reference_text.value
        
        try:
            # 音声生成
            with self.output_widget:
                clear_output(wait=True)
                self.sample_rate, self.audio_data = self.tts.text_to_speech(**params)
            
            # 音声を再生
            with self.audio_widget:
                clear_output(wait=True)
                display(widgets.Audio(data=self.audio_data, rate=self.sample_rate, autoplay=True))
            
            self.status_widget.value = "<b>ステータス:</b> <span style='color:green'>音声の生成に成功しました</span>"
            self.save_btn.disabled = False
            self.download_btn.disabled = False
            self.play_btn.disabled = False
        except Exception as e:
            self.status_widget.value = f"<b>ステータス:</b> <span style='color:red'>エラー: {str(e)}</span>"
            self.save_btn.disabled = True
            self.download_btn.disabled = True
            self.play_btn.disabled = True
    
    def save_audio(self, btn):
        """生成された音声をファイルに保存します"""
        if self.audio_data is None or self.sample_rate is None:
            self.status_widget.value = "<span style='color:red'>エラー: 先に音声を生成してください</span>"
            return
        
        filename = self.filename_input.value
        
        try:
            sf.write(filename, self.audio_data, self.sample_rate)
            self.status_widget.value = f"<b>ステータス:</b> <span style='color:green'>音声を {filename} に保存しました</span>"
        except Exception as e:
            self.status_widget.value = f"<b>ステータス:</b> <span style='color:red'>保存エラー: {str(e)}</span>"
    
    def play_audio(self, btn):
        """最後に生成した音声を再生します"""
        if self.audio_data is None or self.sample_rate is None:
            self.status_widget.value = "<span style='color:red'>エラー: 先に音声を生成してください</span>"
            return
        
        with self.audio_widget:
            clear_output(wait=True)
            display(widgets.Audio(data=self.audio_data, rate=self.sample_rate, autoplay=True))
    
    def download_audio(self, btn):
        """生成された音声をダウンロードします"""
        if self.audio_data is None or self.sample_rate is None:
            self.status_widget.value = "<span style='color:red'>エラー: 先に音声を生成してください</span>"
            return
        
        # WAVファイルを一時的にメモリに保存
        filename = self.filename_input.value
        wav_io = io.BytesIO()
        sf.write(wav_io, self.audio_data, self.sample_rate, format='WAV')
        wav_io.seek(0)
        
        # WAVデータをBase64エンコード
        audio_base64 = base64.b64encode(wav_io.read()).decode('utf-8')
        
        # ダウンロードリンクを作成
        download_link = f'<a href="data:audio/wav;base64,{audio_base64}" download="{filename}" target="_blank">クリックして音声をダウンロード</a>'
        self.download_link_widget.value = download_link
        
        self.status_widget.value = "<b>ステータス:</b> <span style='color:green'>ダウンロードリンクを生成しました</span>"

# アプリのインスタンスを作成
app = FishSpeechTTSApp()